In [ ]:
!git clone https://github.com/WorkInTheDark/FairytaleQA_QAG_System.git
%cd ./FairytaleQA_QAG_System/

Mounted at /gdrive


change the directory to the folder with separate story data

In [ ]:

dir = "./split_for_training/train"
split_version = 'train'

# dir = "./split_for_training/test"
# split_version = 'test'

# dir = "./split_for_training/val"
# split_version = 'val'


In [ ]:
import pandas as pd
import os
import glob
from pathlib import Path
from math import isnan
import csv
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
def is_nan(x):
    return isinstance(x, float) and isnan(x)

In [ ]:
total_data = {}
all_story_data = {}
all_question_data = {}
all_AC_input_data = {}
all_QC_input_data = {}
all_A_output_data = {}

all_A1_output_data = {}
all_A2_output_data = {}
all_Q_input_data = {}
all_C_input_data = {}
all_spreadsheet_data = {}
all_QS_input_data = {}
all_CA_input_data = {}
all_CQA_input_data = {}
all_QAC_input_data = {}
all_AC_ranking_input_data = {}
all_QA_bias_analysis = {}

all_QA_attribute = {}
all_QA_ex_im = {}
train_count = 0


In [ ]:
name_set = []
csv_files = []

cnt_files = 0
for root, dirs, files in os.walk(dir):
  if files == []:
    continue
  cnt_files += len(files)
  for i in files:
    if i.replace("-questions.csv", "").replace("-story.csv", "") not in name_set:
      name_set.append( i.replace("-questions.csv", "").replace("-story.csv", "") )
      csv_files.append(root + "/" + i.replace("-questions.csv", "").replace("-story.csv", "") + "-story.csv" )
      

print("====")
print(len(name_set), len(csv_files))
print("cnt_files: ", cnt_files)

assert len(name_set) == len(csv_files), print( len(name_set), len(csv_files) )
print("# of books: ", len(name_set))


====
232 232
cnt_files:  464
# of books:  232


In [ ]:
cnt = 0
cnt_q = 0


for file_name in csv_files:

  key = file_name.split("/")[-1].replace("-questions.csv", "").replace("-story.csv", "")
  

  if key not in all_question_data:
    all_question_data[key] = []
    
  if key not in all_AC_input_data:
    all_AC_input_data[key] = []
  
  if key not in all_Q_input_data:
    all_Q_input_data[key] = []
  
  if key not in all_C_input_data:
    all_C_input_data[key] = []
  
  if key not in all_story_data:
    all_story_data[key] = []
  
  if key not in all_QC_input_data:
    all_QC_input_data[key] = []
  
  if key not in all_A_output_data:
    all_A_output_data[key] = []

  if key not in all_A1_output_data:
    all_A1_output_data[key] = []
  
  if key not in all_A2_output_data:
    all_A2_output_data[key] = []
  
  if key not in all_spreadsheet_data:
    all_spreadsheet_data[key] = []

  if key not in all_QS_input_data:
    all_QS_input_data[key] = []

  if key not in all_CA_input_data:
    all_CA_input_data[key] = []

  if key not in all_CQA_input_data:
    all_CQA_input_data[key] = []
  
  if key not in all_QAC_input_data:
    all_QAC_input_data[key] = []
  
  if key not in all_AC_ranking_input_data:
    all_AC_ranking_input_data[key] = []
  
  if key not in all_QA_attribute:
    all_QA_attribute[key] = []

  if key not in all_QA_ex_im:
    all_QA_ex_im[key] = []
  
  if key not in all_QA_bias_analysis:
    all_QA_bias_analysis[key] = []
    
  print(key)
  print(file_name)

  total_data[key] = []

  story_data = pd.read_csv(file_name).to_dict('index') 
  question_data = pd.read_csv( file_name.replace("-story.csv", "-questions.csv") ).to_dict('index') 

  story_data_dict = {}

  full_story = ""

  for i in story_data:

    if is_nan(story_data[i]['text']):
      continue

    try: 

      temp_data = story_data[i]['text'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower()
      temp_data = ' '.join(temp_data.split())

      doc = nlp(temp_data)
      tokenized = [token.text for token in doc]
      temp_data = ' '.join(tokenized)

      story_data_dict[int(story_data[i]['section'])] = temp_data
      all_story_data[key].append( temp_data )
      full_story += temp_data + ' <SEP> '

      # <<<DEPRECATED>>> parsing code
      # story_data_dict[int(story_data[i]['section'])] = story_data[i]['text'].replace('\n','').replace('\r','').lower()
      # all_story_data[key].append(story_data[i]['text'].replace('\n','').replace('\r','').lower())
      # full_story += story_data[i]['text'].replace('\n','').replace('\r','').lower() + ' <SEP> '


    except:
      print("---", int(story_data[i]['section']), story_data[i] )

  # REMOVE THE LAST <SEP>
  full_story = full_story[:-6]

  for i in question_data:
    # it is possible one question is related to multiple sections 
    ##print("xx", question_data[i])
    
    text_section_list = str(question_data[i]['cor_section']).split(",")
    total_text = ""
    if len(text_section_list) == 1:

        question_data[i]['text'] = story_data_dict[int(question_data[i]['cor_section'])]
        total_text = story_data_dict[int(question_data[i]['cor_section'])]
    else:
        
        for j in text_section_list:
            total_text += story_data_dict[int(j)].strip() + ' '

        question_data[i]['text'] = total_text
    

    question = question_data[i]['question'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' ?') + ' ?'
    attribute = question_data[i]['attribute1'].replace('\n', '').lower()
    ex_im = question_data[i]['ex-or-im1'].replace('\n', '').lower()

    # there's only one groundtruth answer
    if split_version == 'train':
      answer1 = question_data[i]['answer1'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'
      

      doc = nlp(answer1)
      tokenized = [token.text for token in doc]
      answer1 = ' '.join(tokenized)
      

      all_question_data[key].append( total_text + ' </s> ' + question + ' </s> ' + answer1 )

      all_A_output_data[key].append( answer1 )

      all_QA_attribute[key].append( question + ' <SEP> ' + answer1 + ' <SEP> ' + attribute )
      all_QA_ex_im[key].append( question + ' <SEP> ' + answer1 + ' <SEP> ' + ex_im )
      all_QA_bias_analysis[key].append( total_text + ' <SEP> ' + question + ' <SEP> ' + answer1 + ' <SEP> ' + attribute + ' <SEP> ' + ex_im)
    

    # there are two groundtruth answers
    if split_version == 'not_train':
      answer1 = question_data[i]['answer1'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'
      answer2 = question_data[i]['answer4'].replace('\n',' ').replace('\r',' ').replace('\t',' ').replace('\s',' ').replace('\t+',' ').replace('\s+',' ').lower().strip(' .') + ' .'


      doc = nlp(answer1)
      tokenized = [token.text for token in doc]
      answer1 = ' '.join(tokenized)

      doc = nlp(answer2)
      tokenized = [token.text for token in doc]
      answer2 = ' '.join(tokenized)


      all_question_data[key].append( total_text + ' </s> ' + question + ' </s> ' + answer1 + ' </s> ' + answer2 )
      all_A_output_data[key].append( answer1 + ' </s> ' + answer2 )

      all_A1_output_data[key].append( answer1 )
      all_A2_output_data[key].append( answer2 )

      all_QA_attribute[key].append( question + ' <SEP> ' + answer1 + ' </s> ' + answer2 + ' <SEP> ' + attribute )
      all_QA_ex_im[key].append( question + ' <SEP> ' + answer1 + ' </s> ' + answer2 + ' <SEP> ' + ex_im )
      all_QA_bias_analysis[key].append( total_text + ' <SEP> ' + question + ' <SEP> ' + answer1 + ' </s> ' + answer2 + ' <SEP> ' + attribute + ' <SEP> ' + ex_im)
    
    all_AC_input_data[key].append( answer1 + ' </s> ' + total_text )

    all_Q_input_data[key].append( question )
    all_C_input_data[key].append( total_text )

    all_QC_input_data[key].append( question + ' <SEP> ' + total_text )

    all_QS_input_data[key].append( question + ' <SEP> ' + full_story )
  
    all_CA_input_data[key].append( total_text + ' <SEP> ' + answer1 )
    all_CQA_input_data[key].append( total_text + ' ' + question + ' <SEP> ' + answer1 )
    
    all_AC_ranking_input_data[key].append( answer1 + ' <SEP> ' + total_text )
    all_QAC_input_data[key].append( question + ' <SEP> ' + answer1 + ' <SEP> ' + total_text )

    all_spreadsheet_data[key].append( [ total_text, question, answer1 ] )

    # remove nan key-value pairs in current instance in the dictionary
    new_data_i = {}
    for k in question_data[i].keys():
        if not is_nan(question_data[i][k]):
            new_data_i[k] = question_data[i][k]

    total_data[key].append(new_data_i)

    cnt_q += 1
  cnt += 1


print("# stories: ", cnt)
print("# question: ", cnt_q)




#### Parse and store into different format

In [ ]:
!mkdir ./pre-processed_data

data format to directly fine-tune a BART Question Answering model 

In [ ]:
!mkdir ./pre-processed_data/BART_QA


In [ ]:
if split_version == 'val':

  f1 = open("./pre-processed_data/BART_QA/val.source", "w")
  f2 = open("./pre-processed_data/BART_QA/val_two_answer.target", "w")

if split_version == 'test':

  f1 = open("./pre-processed_data/BART_QA/test.source", "w")
  f2 = open("./pre-processed_data/BART_QA/test_two_answer.target", "w")

for key in name_set:
  for i in all_QC_input_data[key]:
    f1.write( i.replace('\n','').replace('\r','') + '\n' )
  for i in all_A_output_data[key]:
    f2.write( i.replace('\n','').replace('\r','') + '\n' )

f1.close()
f2.close()



data format to fine-tune a BART Question Generation model 

In [ ]:
!mkdir ./pre-processed_data/BART_QG

In [ ]:

if split_version == 'val':

  f1 = open(folder_dir + "./pre-processed_data/BART_QG/val.source", "w")
  f2 = open(folder_dir + "./pre-processed_data/BART_QG/val.target", "w")

if split_version == 'test':

  f1 = open(folder_dir + "./pre-processed_data/BART_QG/test.source", "w")
  f2 = open(folder_dir + "./pre-processed_data/BART_QG/test.target", "w")

for key in name_set:
  for i in all_AC_ranking_input_data[key]:
    f1.write( i + '\n' )
  for i in all_Q_input_data[key]:
    f2.write( i + '\n' )

f1.close()
f2.close()


data format to be used for training an answer ranking model. need to be combined with generated QA-pairs 

In [ ]:
!mkdir ./pre-processed_data/Ranking_QA

In [ ]:

f4 = open("./pre-processed_data/Ranking_QA/train_groundtruth_QAC.txt", "w")

for key in name_set:

    for i in all_QAC_input_data[key]:
        f4.write( i + '\n' )
        
f4.close()

data format for generating QA-pairs with our system, each line is a section

In [ ]:
!mkdir ./pre-processed_data/Ranking_QA/QA_generation
!mkdir ./pre-processed_data/Ranking_QA/QA_generation/test
!mkdir ./pre-processed_data/Ranking_QA/QA_generation/val


In [ ]:
if split_version == 'val':
  for key in name_set:
    f = open("./pre-processed_data/Ranking_QA/QA_generation/val" + "/sections-" + key + ".txt", "w+")
    for i in all_story_data[key]:
      f.write(i + '\n')
    f.close()

if split_version == 'test':
  for key in name_set:
    f = open("./pre-processed_data/Ranking_QA/QA_generation/test" + "/sections-" + key + ".txt", "w+")
    for i in all_story_data[key]:
      f.write(i + '\n')
    f.close()


data format for evaluation, use as groundtruth file

In [ ]:
!mkdir ./pre-processed_data/Ranking_QA/Evaluation_Groundtruth
!mkdir ./pre-processed_data/Ranking_QA/Evaluation_Groundtruth/test
!mkdir ./pre-processed_data/Ranking_QA/Evaluation_Groundtruth/val


In [ ]:
if split_version == 'val':

  for key in name_set:
    f = open("./pre-processed_data/Ranking_QA/Evaluation_Groundtruth/val/Evaluation_GT_stories-" + key + ".txt", "w+")
    for i in all_question_data[key]:
      f.write( i.replace('\n','').replace('\r','') + '\n' )
    f.close()

if split_version == 'test':

  for key in name_set:
    f = open("./pre-processed_data/Ranking_QA/Evaluation_Groundtruth/test/Evaluation_GT_stories-" + key + ".txt", "w+")
    for i in all_question_data[key]:
      f.write( i.replace('\n','').replace('\r','') + '\n' )
    f.close()

data format for training a baseline C -> Q model

In [ ]:
# f1 = open("/gdrive/MyDrive/NLP_Stage_2/Generation_Project/After_EMNLP/FairyTale/ACL_split/BASELINE_BART_CQ/train.source", "w")
# f2 = open("/gdrive/MyDrive/NLP_Stage_2/Generation_Project/After_EMNLP/FairyTale/ACL_split/BASELINE_BART_CQ/train.target", "w")


# for key in name_set:
#     for i in all_C_input_data[key]:

#         f1.write( i.replace('\n','').replace('\r','') + '\n' )

#     for i in all_Q_input_data[key]:

#         f2.write( i.replace('\n','').replace('\r','') + '\n' )


# f1.close()
# f2.close()


data format to prepare input data for PAQ

In [ ]:
# cnt = 0

# with open("/gdrive/MyDrive/NLP_Stage_2/Generation_Project/After_EMNLP/FairyTale/ACL_split/BASELINE_PAQ/PAQ_input_stories-test.txt", 'wt') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
    
#     for key in name_set:
#       for i in all_story_data[key]:
#         tsv_writer.writerow([cnt, i.replace('\n','').replace('\r','').lower(), "val"])
#         cnt += 1




In [ ]:
# print(cnt)